<a href="https://colab.research.google.com/github/Ram060299/Heart-Sound-Processing-via-xLSTM/blob/main/Unhealthy_HS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

zip_path = '/content/drive/MyDrive/unhealthy.zip'
extract_path = '/content/drive/MyDrive/unhealthy_extracted'

# Extract the zip file
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

Mounted at /content/drive
Files extracted to: /content/drive/MyDrive/unhealthy_extracted


In [ ]:
audio_files = []
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith('.wav'):
            audio_files.append(os.path.join(root, file))

print(f"Found {len(audio_files)} audio files.")

Found 665 audio files.


In [ ]:
!pip install noisereduce
import librosa
import soundfile as sf
import numpy as np
import librosa.display
import noisereduce as nr # The noisereduce module is now available
import matplotlib.pyplot as plt
import os

def preprocess_audio(file_path, output_folder):
    # 1. Load and convert audio to 16kHz
    y, sr = librosa.load(file_path, sr=16000)

    # 2. Trim silence
    y_trimmed, _ = librosa.effects.trim(y)

    # 3. Handle corrupted files (log error and skip)
    try:
        y, sr = librosa.load(file_path)
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return  # Skip the file if there’s an error

    # 4. Remove outliers based on amplitude
    threshold = np.mean(np.abs(y_trimmed)) + 3 * np.std(np.abs(y_trimmed))
    y_no_outliers = y_trimmed[np.abs(y_trimmed) < threshold]

    # 5. Convert stereo to mono
    y_mono = librosa.to_mono(y_no_outliers)

    # 6. Fix duration to 10 seconds
    max_duration = 10.0
    y_fixed_length = librosa.util.fix_length(y_mono, size=int(sr * max_duration))

    # 7. Optional: Plot waveform
    plt.figure(figsize=(10, 4))
    librosa.display.waveshow(y_fixed_length, sr=sr)
    plt.title(f'Waveform of {os.path.basename(file_path)}')
    plt.show()

    # 8. Normalize
    y_normalized = librosa.util.normalize(y_fixed_length)

    # 9. Noise reduction
    y_denoised = nr.reduce_noise(y=y_normalized, sr=sr)

    # 10. Save processed file
    output_path = os.path.join(output_folder, os.path.basename(file_path))
    sf.write(output_path, y_denoised, sr)

   # Check if output directory exists, if not create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        sf.write(output_path, y_denoised, sr)
    except Exception as e:
        print(f"Error saving file {output_path}: {e}")
        return

    print(f"Processed and saved: {output_path}")


In [ ]:
output_folder = '/content/drive/MyDrive/processed_audio'

# Make sure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each audio file automatically
for i, audio_file in enumerate(audio_files):
    print(f"Processing file {i+1}/{len(audio_files)}: {audio_file}")
    preprocess_audio(audio_file, output_folder)

print("All files have been processed.")

NameError: name 'audio_files' is not defined